# Windpower

In [1]:
from ruins.core import build_config

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
import xarray as xr

2022-07-21 08:29:05.565 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


In [2]:
config, dm = build_config()

In [3]:
climate = dm.read('climate')

## windspeed plot

In [4]:
def windspeeds(climate: xr.Dataset, variable: str, rcp: str = 'rcp85', color='green', bgcolor='lightgreen', fig: go.Figure = None, col: int = 1, row: int = 1) -> go.Figure:
    # get the aggregated data
    df = climate.sel(vars=variable).to_dataframe().groupby(pd.Grouper(freq='a')).mean()
    
    # select by RCP scenario
    if rcp is not None:
        data = df[[c for c in df.columns if c.endswith(rcp)]]
    else:
        data = df

    # get the figure
    if fig is None:
        fig = make_subplots(1, 1)

    # build the basic figure
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=np.nanquantile(data.values, 0.95, axis=1), mode='lines', line=dict(color=bgcolor), fill='none', showlegend=False),
        col=col, row=row
    )
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=np.nanquantile(data.values, 0.05, axis=1), mode='lines', line=dict(color=bgcolor), fill='tonexty', showlegend=False),
        col=col, row=row
    )
    fig.add_trace(
        go.Scatter(x=data.mean(axis=1).index, y=data.mean(axis=1), mode='lines', line=dict(color=color, width=2), name=f'{rcp.upper()} mean'),
        col=col, row=row
    )

    # layout
    fig.update_layout(
        **{f'yaxis{row}': dict(title=f'{rcp.upper()}<br>Windspeed [m/s]')}
    )

    return fig

In [5]:
fig = make_subplots(3, 1, shared_xaxes=True, vertical_spacing=0.0)

fig = windspeeds(climate, 'u2', rcp='rcp26', fig=fig, row=1, color='blue', bgcolor='lightblue')
fig = windspeeds(climate, 'u2', rcp='rcp45', fig=fig, row=2)
fig = windspeeds(climate, 'u2', rcp='rcp85', fig=fig, row=3, color='red', bgcolor='red')

fig.update_layout(height=600, xaxis3=dict(title='Year'), legend=dict(orientation='h'))

## windenergy upscaling

In [6]:
from typing import Union, Tuple, List
import numpy as np
from numba import jit

TURBINES = dict(
    e53=(0.8, 53),
    e115=(3, 115),
    e126=(7.5, 126)
)

#@jit(forceobj=True)
def turbine_footprint(turbine: Union[str, Tuple[float, int]], unit: str = 'ha'):
    """Calculate the footprint for the given turbine dimension"""
    if isinstance(turbine, str):
        turbine = TURBINES[turbine]
    mw, r = turbine
    
    # get the area - 5*x * 3*y 
    area = ((5 * r) * (3 * r))   # m2

    if unit == 'ha':
        area /= 10000
    elif unit == 'km2':
        area / 1000000

    # return area, mw
    return area, mw

#@jit
def upscale_windenergy(turbines: List[Union[str, Tuple[float, int]]], specs: List[Tuple[float]], site: float = 396.0) -> np.ndarray:
    """
    Upscale the given turbines to the site.
    Pass a list of turbine definitions (either names or MW, rotor_diameter tuples.).
    The function will apply the specs to the site. The specs can either be absolute number of
    turbines per turbine or relative shares per turbine type.
    Returns a tuple for each turbine type.

    Returns
    -------
    List[Tuple[float, int, float]]
        A list of tuples per turbine type. (n_turbines, total_area, total_mw)
    """
    # check input data
    #if not all([len(spec)==len(turbines) for spec in specs]):
    #    raise ValueError('The number of turbines and the number of specs must be equal.')
    
    # result container
    results = np.ones((len(specs) * len(turbines), len(turbines))) * np.NaN

    # get the area and MW for each used turbine type
    turbine_dims = [turbine_footprint(turbine) for turbine in turbines]

    for i in range(len(specs)):
        for j in range(len(turbines)):
            # get the footprint
            #area, mw = turbine_footprint(turbine, unit='ha')
            area, mw = turbine_dims[j]

            # get the available space and place as many turbines as possible
            n_turbines = int((site * specs[i][j]) / area)
            
            # get the used space and total MW
            used_area = n_turbines * area
            used_mw = n_turbines * mw

            results[i * len(turbines) + j,:] = [n_turbines, used_area, used_mw]

    return results


In [19]:
r = upscale_windenergy(['e53', 'e115', 'e126'], [(1, 0, 0), (0., 1, 0.), (0., 0, 1), (0.47, 0.53, 0),])
r

array([[ 93.    , 391.8555,  74.4   ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [ 19.    , 376.9125,  57.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [  0.    ,   0.    ,   0.    ],
       [ 16.    , 381.024 , 120.    ],
       [ 44.    , 185.394 ,  35.2   ],
       [ 10.    , 198.375 ,  30.    ],
       [  0.    ,   0.    ,   0.    ]])

## Structure Windpower data

In [8]:
from ruins.processing.windpower import load_windpower_data

In [21]:
import warnings
warnings.simplefilter('ignore', category=pd.errors.PerformanceWarning)

from scipy.stats import gaussian_kde
from ruins.core import DataManager

def windpower_distplot(dataManager: DataManager, specs, turbines=['e53', 'e115', 'e126'], site: float = 396.0, fig: go.Figure = None, filter_={}) -> go.Figure:
    """
    """
    if fig is None:
        fig = go.Figure()

    # handle the specs
    if len(specs) == 1 and any([isinstance(s, range) for s in specs]):
        # there is a range definition
        scenarios = []
        for e1 in specs[0]:
            for e2 in specs[1]:
                for e3 in specs[2]:
                    scenarios.append((e1 / 100, e2 / 100, e3 / 100))
    else:
        scenarios = specs

    # upscale the turbines to the site
    power_share = upscale_windenergy(turbines, scenarios)

    # get the data
    df = load_windpower_data(dataManager)

    # apply filters
    for key, val in filter_.items():
        if key == 'year':
            df = df[val]
        elif key == 'rcp':
            df = df.xs(val, level=2, axis=1)
        elif key == 'gcm':
            df = df.xs(val, level=3, axis=1)

    # aggregate everything
    actions = []
    for i in range(0, len(power_share), len(turbines)):
        data = None
        for j, pw in enumerate(power_share):
            pass
    return fig


# define properties
RCP = 'rcp85'

# get turbines
specs = ['e53', 'e115', 'e126']
#turbine = upscale_windenergy(['e53', 'e115', 'e126'], [(1, 0, 0), (0, 1, 0), (0.5, 0, 0.5)])
#scenario = [(i / 100, 0 , 1 - (i / 100)) for i in range(0, 100, 5)]
scenario = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
turbine = upscale_windenergy(specs, scenario)

# load
df = load_windpower_data(dm)['2075':'2095']

# aggregate
actions = []
for i in range(0, len(turbine), len(specs)):
    data = None
    for j, spec in enumerate(specs):
        chunk = df[spec.upper(), RCP]# .mean(axis=1)  # mean value per turbine
        chunk *= turbine[i + j][0]  # multiply with the number of turbines
        if data is None:
            #data = pd.DataFrame(data={spec: chunk.values}, index=chunk.index)
            data = chunk
        else:
            # data[spec] = chunk.values
            data = pd.merge(data, chunk, left_index=True, right_index=True, how='outer')
    actions.append(data)


# Plotting
# go for the plot
# import plotly.figure_factory as ff
# fig = ff.create_distplot([a.sum(axis=1) for a in actions], [f'{i / len(actions) * 100:.0f}%' for i in range(len(actions))], show_hist=False, show_rug=False)
import plotly.graph_objects as go

fig = go.Figure()
for i, action in enumerate(actions):
    y = action.sum(axis=1).values
    x = np.linspace(y.min(), y.max(), 100)
    kde = gaussian_kde(y)(x)

    fig.add_trace(go.Scatter(x=x, y=kde, mode='lines', line=dict(color='blue', width=0.), fill='tozeroy', showlegend=False, name=f'{i * 5}% E53'))

fig.update_layout(width=500, height=500, template='plotly_white')


In [24]:
df[slice('2080', '2100')]
df

,E115,E126,E53,E115,E126,E53,E115,E126,E53,E115,...,E53,E115,E126,E53,E115,E126,E53,E115,E126,E53
,rcp85,rcp85,rcp85,rcp85,rcp85,rcp85,rcp85,rcp85,rcp85,rcp85,...,rcp85,rcp26,rcp26,rcp26,rcp45,rcp45,rcp45,rcp85,rcp85,rcp85
,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,CLMcom-ETH,...,DWD,DWD,DWD,DWD,DWD,DWD,DWD,DWD,DWD,DWD
,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,COSMO-crCLIM-v1-1,...,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018,EPISODES2018
,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r1i1p1.rcp85_E115,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r1i1p1.rcp85_E126,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r1i1p1.rcp85_E53,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r2i1p1.rcp85_E115,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r2i1p1.rcp85_E126,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r2i1p1.rcp85_E53,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r3i1p1.rcp85_E115,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r3i1p1.rcp85_E126,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r3i1p1.rcp85_E53,CLMcom-ETH.NorESM1-M.COSMO-crCLIM-v1-1.r1i1p1.rcp85_E115,...,DWD.M-MPI-ESM-LR.EPISODES2018.r3i1p1.rcp85_E53,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp26_E115,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp26_E126,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp26_E53,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp45_E115,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp45_E126,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp45_E53,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp85_E115,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp85_E126,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp85_E53
time,,,,,,,,,,,,,,,,,,,,,
2075-12-31,13570.282970,23930.081423,3172.491555,13262.157882,22953.889486,3077.173916,12647.891422,21639.927121,2922.215076,12651.043858,...,2970.416956,12334.059287,21214.202702,2844.977154,13012.847826,22823.270511,3036.594474,12874.711684,22514.266840,2998.724655
2076-12-31,11625.060656,19617.901196,2664.688164,12189.252064,20405.018138,2774.370842,11651.494208,19342.255278,2643.857207,13663.618199,...,3126.300761,13037.770233,22353.553357,3020.640061,14389.877925,25708.217735,3395.333794,11928.043797,20246.654853,2735.939793
2077-12-31,12467.658802,21548.290878,2882.311049,11847.658598,20054.991100,2712.871295,12768.445407,22697.268392,2995.543007,13296.235438,...,3212.818868,14345.740546,25412.665649,3368.869611,13171.387196,22427.139597,3033.753068,13044.420346,23022.448725,3065.670426
2078-12-31,13689.782860,24656.345744,3229.997776,12929.392814,22562.463023,3016.922907,14324.471001,25847.013080,3397.920417,14376.450798,...,3249.200877,13451.142615,23358.468691,3120.417338,14314.173341,25722.998267,3387.211483,14131.565092,24858.900432,3312.654698
2079-12-31,13127.673529,22698.192731,3038.178700,11789.056400,20154.066975,2711.541883,13577.501196,24112.268607,3181.814330,13967.041972,...,3147.085371,13195.873124,23586.772558,3088.840236,12331.298877,21076.656412,2837.115970,14607.596804,25597.155882,3410.106578
